### <div style="text-align: justify;">PROJETO: DETECÇÃO DE FRAUDES NO TRÁFEGO DE CLIQUES EM PROPAGANDAS DE APLICAÇÕES MOBILE - TALKINGDATA</div>

### DESCRIÇÃO DOS DADOS

Dados de Treino:

Cada linha dos dados de treinamento contém um registro de cliques, com os seguintes recursos.

- ip --> **ip**: ip endereço de clique.
- app --> **app**: id de aplicativo para marketing.
- device --> **dispositivo**: identificação do tipo de dispositivo do celular do usuário (por exemplo, iphone 6 plus, iphone 7, huawei mate 7, etc.)
- os --> **os**: id versão do sistema operacional do celular do usuário
- channel --> **canal**: id do canal da editora de anúncios móveis
- click-time --> **horario_clique**: fuso de hora em hora (UTC)
- attributed-time --> **horario_download**: se o usuário baixar o aplicativo para depois de clicar em um anúncio, esta é a hora do download do aplicativo
- is-attributed --> **classe**: o alvo a ser previsto, indicando que o aplicativo foi baixado (não fraude = 1) ou não baixado (fraude = 0).

Os dados do teste são semelhantes, com as seguintes diferenças:

- click-id: referência para fazer previsões
- is-attributed: não incluído

### PRÉ-PROCESSAMENTO 

#### CARREGAR BIBLIOTECAS

In [1]:
# DESATIVAR MENSAGENS DE WARNINGS (warn = -1) E ATIVAR (warn = 0)
options(warn = -1)

In [2]:
# CARREGAR BIBLIOTECAS 
suppressPackageStartupMessages({
library(data.table)
library(dplyr)  
library(readr)
})

#### DADOS TREINO

In [3]:
gc()
memory.limit(9999999999)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658410,35.2,1370228,73.2,1370228,73.2
Vcells,46136483,352.0,69642036,531.4,46711952,356.4


[1] 1e+10

In [4]:
# # CARREGAR DADOS DE TREINO DISPONIBILIZADOS NO KAGGLE 
train_csv <- fread("train.csv", encoding = "UTF-8")

In [5]:
# RENOMEAR COLUNAS DATASET
names(train_csv) <- c("ip", "app", "dispositivo", "so", "canal", "horario_clique", "horario_download", "classe")

In [6]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,673518,36,1370228,73.2,1370228,73.2
Vcells,1062855864,8109,1386691235,10579.7,1062999608,8110.1


#### COLETA  DAS  AMOSTRAS E FORMAÇÃO DA BASE DE DADOS

<p>
<div style="text-align: justify;">
Formação da base de dados a partir de amostras coletadas a partir da base de dados disponibilizada no Kaggle. Utilizou-se a função sample_n() para coletar amostras de mesma quantidade para as duas classes. No total, a base formada para este experimento foi de 240.000 observações balanceadas.
</div>
</p>

In [7]:
set.seed(1908)

In [8]:
# 1 - COLETAR AMOSTRA DE DADOS DE CLIQUES QUE BAIXARAM O APLICATIVO 
treino_baixado <- sample_n(train_csv[train_csv$classe == 1,], size = 120000)
glimpse(treino_baixado)

Rows: 120,000
Columns: 8
$ ip               <int> 360958, 4414, 101455, 363652, 277861, 73933, 79827, 1~
$ app              <int> 45, 127, 9, 10, 19, 12, 19, 35, 14, 35, 96, 29, 10, 1~
$ dispositivo      <int> 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 6, 1, 1, 1, 1, 0, 1, 0,~
$ so               <int> 26, 24, 19, 36, 24, 13, 0, 16, 43, 13, 29, 13, 11, 30~
$ canal            <int> 5, 213, 127, 113, 213, 178, 347, 21, 460, 21, 213, 21~
$ horario_clique   <dttm> 2017-11-09 06:06:01, 2017-11-07 06:26:01, 2017-11-09~
$ horario_download <dttm> 2017-11-09 06:06:07, 2017-11-07 06:28:18, 2017-11-09~
$ classe           <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,~


In [9]:
# 2 - COLETAR AMOSTRA DE DADOS DE CLIQUES QUE NÃO BAIXARAM O APLICATIVO 
treino_naobaixado <- sample_n(train_csv[train_csv$classe == 0,], size = 120000)
glimpse(treino_naobaixado)

Rows: 120,000
Columns: 8
$ ip               <int> 153289, 42186, 65295, 105475, 1206, 81574, 70327, 118~
$ app              <int> 15, 9, 64, 15, 1, 8, 3, 3, 8, 12, 26, 20, 14, 6, 6, 1~
$ dispositivo      <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,~
$ so               <int> 13, 8, 32, 20, 19, 19, 19, 20, 19, 19, 6, 19, 12, 41,~
$ canal            <int> 245, 334, 459, 3, 349, 145, 489, 205, 145, 19, 477, 2~
$ horario_clique   <dttm> 2017-11-07 11:44:09, 2017-11-07 23:53:39, 2017-11-07~
$ horario_download <dttm> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, ~
$ classe           <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,~


In [10]:
# 3 - COMBINAR OS DADOS PARA CRIAR UM DATASET BALANCEADO
df_talking_data <- rbind(treino_baixado, treino_naobaixado)
df_talking_data <- df_talking_data[sample(1:nrow(df_talking_data)), ]
glimpse(df_talking_data)

Rows: 240,000
Columns: 8
$ ip               <int> 159048, 191117, 10833, 113682, 6534, 130391, 286692, ~
$ app              <int> 39, 18, 20, 6, 9, 10, 190, 19, 12, 19, 8, 12, 3, 10, ~
$ dispositivo      <int> 1, 1, 1, 1, 1, 1, 1, 129, 1, 0, 1, 1, 1, 1, 1, 0, 1, ~
$ so               <int> 14, 10, 10, 17, 13, 61, 18, 24, 32, 24, 19, 13, 19, 7~
$ canal            <int> 213, 376, 259, 459, 489, 113, 213, 213, 259, 213, 145~
$ horario_clique   <dttm> 2017-11-07 02:49:38, 2017-11-07 08:10:07, 2017-11-07~
$ horario_download <dttm> 2017-11-07 02:54:44, NA, NA, NA, NA, 2017-11-07 02:4~
$ classe           <int> 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,~


In [11]:
# SALVAR DATASET
write_csv(df_talking_data, "df-talking-data.csv")

Nas etapas acima foram realizadas:

- Carregamento do dataset original train.csv do Kaggle;
- <div style="text-align: justify;">Coleta de uma amostra de dados de cliques que baixaram e não baixaram o aplicativo. Foram coletados 120.000 observações de cada classe a partir do dataset train.csv;</div>
- Coleta de amostra dos dados de cliques (treino_baixado) que baixaram o aplicativo;
- Coleta de amostra de dados de cliques (treino_naobaixado) que não baixaram o aplicativo;
- Por fim, a união das amostras e formação do dataset (df_talking_data.csv) utilizado no experimento.